In [18]:
import numpy as np
import networkx as nx
import matplotlib as plt

from fairpair import FairPairGraph, Distributions, random_list_to_pairs, RandomSampling, ProbKnockoutSampling, OversampleMinority

In [19]:
# Initialize the graph with nodes labeled majority or minority
G = FairPairGraph()
G.generate_groups(500, 200)

for i in range(298, 303):
    print(G.nodes(data=True)[i])

{'minority': False}
{'minority': False}
{'minority': True}
{'minority': True}
{'minority': True}


In [20]:
# Add randomization to the group label attribution
G = FairPairGraph()
G.add_nodes_from(np.arange(500))
G.label_minority(200, random=True)

for i in range(5):
    print(G.nodes(data=True)[i])

{'minority': False}
{'minority': True}
{'minority': True}
{'minority': False}
{'minority': False}


In [21]:
# Access the minority subgraph (read-only)
G.minority

In [22]:
# Access the majority nodes
G.majority_nodes[:20]

[0, 3, 4, 5, 6, 7, 10, 14, 15, 16, 17, 19, 20, 22, 25, 26, 27, 28, 29, 30]

In [23]:
# Assign scores to the majority and minority respectively
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.4) # give a disadvantage to the minority
G.nodes(data=True)[11]

{'minority': True, 'score': 0.5862874650450427}

In [24]:
# Compare pairs using the BTL-model
print('before:', G.nodes[1]['score'], G.nodes[5]['score'], G.edges(data=True))
G.compare_pair(1, 5, k=30)
print('after:', G.nodes[1]['score'], G.nodes[5]['score'], G.edges(data=True))

before: 0.4874004272048572 0.35483285293025796 []
after: 0.4874004272048572 0.35483285293025796 [(1, 5, {'wins': 10, 'weight': 0.3333333333333333}), (5, 1, {'wins': 20, 'weight': 0.6666666666666667})]


In [25]:
# Sum up additional comparisons with already existing ones
G.compare_pair(1, 5, k=30)
print(G.nodes[1]['score'], G.nodes[5]['score'], G.edges(data=True))

0.4874004272048572 0.35483285293025796 [(1, 5, {'wins': 24, 'weight': 0.4}), (5, 1, {'wins': 36, 'weight': 0.6})]


In [26]:
random_list_to_pairs(G, range(10))

[(3, 0), (2, 7), (8, 1), (9, 5), (4, 6)]

## Random Sampling

In [27]:
# Generate a new Graph to test random sampling
G = FairPairGraph()
G.generate_groups(20, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.1) # give a disadvantage to the minority
G.edges(data=True)

OutEdgeDataView([])

In [28]:
sampling = RandomSampling(G)
sampling.apply(iter = 8)
print('minority success rate:', round(np.mean([rate for node, rate in G.minority.success_rates]), 2))
print('majority success rate:', round(np.mean([rate for node, rate in G.majority.success_rates]), 2))

minority success rate: 0.44
majority success rate: 0.47


In [29]:
pos=nx.drawing.layout.spring_layout(G)
colors = ['red' if node[1]['minority'] else 'blue' for node in G.nodes(data=True)]
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
#nx.drawing.nx_pylab.draw_networkx(G, pos, node_color=colors, with_labels=False, node_size=50, edge_color=weights, width=2, edge_cmap=plt.cm.Blues)

In [30]:
for node in G.nodes(data=True):
    node[1]['minority'] = bool(node[1]['minority']) # convert from numpy
nx.write_gexf(G, './data/fairpairgraphtest.gexf')

## ProbKnockout Sampling

In [31]:
# Generate a new Graph to test probKnockout sampling
G = FairPairGraph()
G.generate_groups(20, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.1) # give a disadvantage to the minority

In [32]:
sampling = ProbKnockoutSampling(G)
sampling.apply(iter = 8)
print('minority success rate:', round(np.mean([rate for node, rate in G.minority.success_rates]), 2))
print('majority success rate:', round(np.mean([rate for node, rate in G.majority.success_rates]), 2))

minority success rate: 0.2
majority success rate: 0.07


## OversampleMinority

In [33]:
# Generate a new Graph to test OversampleMinority
G = FairPairGraph()
G.generate_groups(20, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.1) # give a disadvantage to the minority

In [34]:
sampling = OversampleMinority(G)
sampling.apply(iter=8, n=5)
print('minority success rate:', round(np.mean([rate for node, rate in G.minority.success_rates]), 2))
print('majority success rate:', round(np.mean([rate for node, rate in G.majority.success_rates]), 2))

minority success rate: 0.44
majority success rate: 0.42
